In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('data/twowk_school_window_data.csv')
immersive_data = pd.read_csv('data/twowk_school_immersive_data.csv')
data.head()

,MEASUREMENT_TIME,ID_INPUT,VALUE,PRIVATE_DATA
0,2022-01-03 17:59:23.547,7,cmd.exe,0
1,2022-01-03 17:59:28.589,7,explorer.exe,0
2,2022-01-03 17:59:59.817,7,ApplicationFrameHost.exe,0
3,2022-01-03 18:00:00.818,7,explorer.exe,0
4,2022-01-03 18:00:15.934,7,chrome.exe,0


In [5]:
data.iloc[0].VALUE

'cmd.exe'

In [6]:
def drop_dups(df):
    bool_indexer = [True] * len(df)
    for i in range(len(df) - 1): #-1 to avoid IndexOutOfBoundsError
        curr_win = df.iloc[i].window
        next_win = df.iloc[i+1].window
        #if the next window is the same then add index to indeces to drop list
        if curr_win == next_win:
            bool_indexer[i+1] = False
    return df[bool_indexer]

In [7]:
def clean_data(path):
    df = pd.read_csv(path)
    #if path refers to a window csv
    if df.VALUE.dtype == 'object':
        #drop unecessary cols and rename the remaining cols
        df = df.drop(['ID_INPUT','PRIVATE_DATA'], axis = 1)
        df.columns = ['time','window']

        #change time col to datetime object
        df['time'] = pd.to_datetime(df['time'])

        #handle back-to-back same windows
        clean_df = drop_dups(df)
    return clean_df
    

In [8]:
#convert days spent on window to minutes spent on window
def convert_time(time):
    return round(float((time.total_seconds()/60)), 2)

In [9]:
df = clean_data('data/twowk_school_window_data.csv')

In [32]:
#Extract time(in min) spent within each app
#returns pd.Series with time spent on window 
#last window will have 0 time spent
def calc_time(data):
    
    data['tvalue'] = data.time
    data['delta'] = (data['tvalue'] - data['tvalue'].shift()).fillna(pd.Timedelta(seconds=0))

    time_col = list(data['delta'][1:]) + [pd.Timedelta(seconds=0)]
    time_spent = pd.Series(time_col)
    time_spent.index = data.window
    
    return time_spent


In [38]:
time_spent_col = calc_time(clean_data('data/twowk_school_window_data.csv'))

In [41]:
df['time_spent'] = time_spent_col.values
df

,time,window,time_spent
0,2022-01-03 17:59:23.547,cmd.exe,00:00:05.042000
1,2022-01-03 17:59:28.589,explorer.exe,00:00:31.228000
2,2022-01-03 17:59:59.817,ApplicationFrameHost.exe,00:00:01.001000
3,2022-01-03 18:00:00.818,explorer.exe,00:00:15.116000
4,2022-01-03 18:00:15.934,chrome.exe,00:29:52.895000
...,...,...,...
1491,2022-01-18 17:28:23.985,Discord.exe,00:00:03.034000
1492,2022-01-18 17:28:27.019,python.exe,00:00:03.022000
1493,2022-01-18 17:28:30.041,explorer.exe,00:00:01.008000
1494,2022-01-18 17:28:31.049,SearchApp.exe,00:00:01.010000
